<a href="https://colab.research.google.com/github/Vaibhavs10/scratchpad/blob/main/gemma_tokenizer_compare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install git+https://github.com/google/gemma_pytorch.git git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/google/gemma_pytorch.git to /tmp/pip-req-build-pttwm4i1
  Running command git clone --filter=blob:none --quiet https://github.com/google/gemma_pytorch.git /tmp/pip-req-build-pttwm4i1
  Resolved https://github.com/google/gemma_pytorch.git to commit 2e90350c992b01db56e211bc1db80b03dcb7375c
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-fl3jbnil
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-fl3jbnil
  Resolved https://github.com/huggingface/transformers.git to commit dcdda5324bcc7a750b5e40e11dd795442204ff27
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 56.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using

In [2]:
from gemma.tokenizer import Tokenizer

In [3]:
original = Tokenizer("tokenizer-2.model")

In [4]:
original.encode("whatever", bos=False)

[74498]

In [5]:
from transformers import GemmaTokenizer, AutoTokenizer

In [15]:
t_fast = AutoTokenizer.from_pretrained("gg-hf/recurrentgemma-9b-hf", revision="pr/2")
t_slow = GemmaTokenizer.from_pretrained("gg-hf/recurrentgemma-9b-hf", revision="pr/2")

# t_fast = AutoTokenizer.from_pretrained("google/recurrentgemma-2b", revision="main")
# t_slow = GemmaTokenizer.from_pretrained("google/recurrentgemma-2b", revision="main")

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [7]:
from datasets import load_dataset

In [8]:
xnli = load_dataset("xnli", "all_languages", split="validation")

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5010 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2490 [00:00<?, ? examples/s]

In [9]:
p = next(iter(xnli["premise"]))

In [16]:
for lang, text in p.items():
    encoded_original = original.encode(text, bos=True, eos=False)
    encoded_fast = t_fast.encode(text)
    encoded_slow = t_slow.encode(text)
    assert encoded_fast == encoded_original, f"Fast encode error: {lang} - {text}"
    assert encoded_slow == encoded_original, f"Slow encode error: {lang} - {text}"
    decoded = original.decode(encoded_original)
    decoded_fast = t_fast.decode(encoded_fast, skip_special_tokens=True)
    decoded_slow = t_slow.decode(encoded_slow, skip_special_tokens=True)
    assert decoded_fast == decoded, f"Fast decode error: {lang} - {text}"
    assert decoded_slow == decoded, f"Slow decode error: {lang} - {text}"

In [11]:
from tqdm import tqdm

In [17]:
def verify(lang, text):
    encoded_original = original.encode(text, bos=True, eos=False)
    encoded_fast = t_fast.encode(text)
    encoded_slow = t_slow.encode(text)
#     assert encoded_fast == encoded_original, f"Fast encode error: {lang} - {text}"
    assert encoded_slow == encoded_original, f"Slow encode error: {lang} - {text}"
    decoded = original.decode(encoded_original)
    decoded_fast = t_fast.decode(encoded_fast, skip_special_tokens=True)
    decoded_slow = t_slow.decode(encoded_slow, skip_special_tokens=True)
#     assert decoded_fast == decoded, f"Fast decode error: {lang} - {text}"
    assert decoded_slow == decoded, f"Slow decode error: {lang} - {text}"

In [18]:
for p in tqdm(xnli["premise"]):
    for lang, text in p.items():
        verify(lang, text)

100%|██████████| 2490/2490 [01:41<00:00, 24.52it/s]


In [ ]:
ds = load_dataset("codeparrot/github-code", streaming=True, split="train")

In [ ]:
next(iter(ds))

{'code': '\'use strict\';\n\nvar clear          = require(\'es5-ext/array/#/clear\')\n  , eIndexOf       = require(\'es5-ext/array/#/e-index-of\')\n  , setPrototypeOf = require(\'es5-ext/object/set-prototype-of\')\n  , callable       = require(\'es5-ext/object/valid-callable\')\n  , d              = require(\'d\')\n  , ee             = require(\'event-emitter\')\n  , Symbol         = require(\'es6-symbol\')\n  , iterator       = require(\'es6-iterator/valid-iterable\')\n  , forOf          = require(\'es6-iterator/for-of\')\n  , Iterator       = require(\'./lib/iterator\')\n  , isNative       = require(\'./is-native-implemented\')\n\n  , call = Function.prototype.call, defineProperty = Object.defineProperty\n  , SetPoly, getValues;\n\nmodule.exports = SetPoly = function (/*iterable*/) {\n\tvar iterable = arguments[0];\n\tif (!(this instanceof SetPoly)) return new SetPoly(iterable);\n\tif (this.__setData__ !== undefined) {\n\t\tthrow new TypeError(this + " cannot be reinitialized");\n\t}

In [ ]:
skipped = 0
iterator = iter(ds)
for _ in tqdm(range(1000)):
    item = next(iterator)
    code = item["code"]
    lang = item["language"]
    if False and item["size"] > 1000:
        skipped += 1
        continue
    verify(lang, code)

100%|██████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:46<00:00, 21.29it/s]


In [ ]:
skipped

0

In [ ]:
len("▁ ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁")

32

In [ ]:
encoded = original.encode(code)
print(encoded)

[2, 809, 8071, 774, 777, 5585, 1730, 108, 809, 6263, 55150, 235265, 37189, 1730, 109, 3249, 3097, 3022, 1236, 612, 108, 139, 773, 591, 108, 141, 235322, 927, 5060, 627, 55150, 235290, 927, 768, 108, 143, 235322, 14789, 5060, 627, 7773, 58367, 2915, 67594, 235290, 55150, 235281, 4881, 627, 235274, 235276, 235276, 40018, 5721, 627, 235274, 235276, 235276, 235281, 2116, 3207, 627, 235276, 235248, 235276, 235248, 235274, 235276, 235276, 235248, 235274, 235276, 235284, 235281, 26271, 145417, 627, 10763, 768, 108, 147, 235322, 2222, 499, 627, 235296, 235276, 235248, 235276, 629, 235276, 235248, 235274, 235276, 235276, 629, 235324, 235276, 235248, 235276, 629, 235274, 235276, 235276, 235248, 235274, 235276, 235276, 629, 235274, 235276, 235276, 235248, 235276, 1271, 235281, 4224, 108, 143, 727, 14789, 235313, 108, 143, 235322, 235259, 235284, 5060, 627, 55150, 235290, 5351, 768, 43629, 193, 108, 143, 235322, 235263, 5060, 627, 55150, 474, 4526, 768, 108, 147, 95661, 235283, 988, 235318, 8071, 

In [ ]:
fast_encoded = t_fast.encode(code)
print(fast_encoded)

[2, 809, 8071, 774, 777, 5585, 1730, 108, 809, 6263, 55150, 235265, 37189, 1730, 109, 3249, 3097, 3022, 1236, 612, 108, 139, 773, 591, 108, 141, 235322, 927, 5060, 627, 55150, 235290, 927, 768, 108, 143, 235322, 14789, 5060, 627, 7773, 58367, 2915, 67594, 235290, 55150, 235281, 4881, 627, 235274, 235276, 235276, 40018, 5721, 627, 235274, 235276, 235276, 235281, 2116, 3207, 627, 235276, 235248, 235276, 235248, 235274, 235276, 235276, 235248, 235274, 235276, 235284, 235281, 26271, 145417, 627, 10763, 768, 108, 147, 235322, 2222, 499, 627, 235296, 235276, 235248, 235276, 629, 235276, 235248, 235274, 235276, 235276, 629, 235324, 235276, 235248, 235276, 629, 235274, 235276, 235276, 235248, 235274, 235276, 235276, 629, 235274, 235276, 235276, 235248, 235276, 1271, 235281, 4224, 108, 143, 727, 14789, 235313, 108, 143, 235322, 235259, 235284, 5060, 627, 55150, 235290, 5351, 768, 43629, 727, 235259, 235284, 235313, 108, 143, 235322, 235263, 5060, 627, 55150, 474, 4526, 768, 108, 147, 95661, 235

In [ ]:
len(encoded), len(fast_encoded)

(265, 268)

In [ ]:
for i, (x, y) in enumerate(zip(encoded, fast_encoded)):
    if x != y:
        print(f"Mismatch at {i}: {x} != {y}")

Mismatch at 134: 193 != 727
Mismatch at 135: 108 != 235259
Mismatch at 136: 143 != 235284
Mismatch at 137: 235322 != 235313
Mismatch at 138: 235263 != 108
Mismatch at 139: 5060 != 143
Mismatch at 140: 627 != 235322
Mismatch at 141: 55150 != 235263
Mismatch at 142: 474 != 5060
Mismatch at 143: 4526 != 627
Mismatch at 144: 768 != 55150
Mismatch at 145: 108 != 474
Mismatch at 146: 147 != 4526
Mismatch at 147: 95661 != 768
Mismatch at 148: 235283 != 108
Mismatch at 149: 988 != 147
Mismatch at 150: 235318 != 95661
Mismatch at 151: 8071 != 235283
Mismatch at 152: 139738 != 988
Mismatch at 153: 13116 != 235318
Mismatch at 154: 17852 != 8071
Mismatch at 155: 94776 != 139738
Mismatch at 156: 81690 != 13116
Mismatch at 157: 37466 != 17852
Mismatch at 158: 10210 != 94776
Mismatch at 159: 235265 != 81690
Mismatch at 160: 2833 != 37466
Mismatch at 161: 188629 != 10210
Mismatch at 162: 82780 != 235265
Mismatch at 163: 713 != 2833
Mismatch at 164: 3626 != 188629
Mismatch at 165: 4791 != 82780
Mismatc

In [ ]:
correct_fast = AutoTokenizer.from_pretrained("google/recurrentgemma-2b", revision="refs/pr/6")

In [ ]:
correct_fast.encode(code) == encoded

True

-----

This is a genuine problem with the fast tokenizer from the PR. A long sequence of spaces is encoded to two different tokens.

In [ ]:
original.decode([168])

'                               '

In [ ]:
original.decode([153])

'                '

In [ ]:
t_fast.decode([168])

'                               '

In [ ]:
t_fast.decode([153])

'                '

In [ ]:
original.encode('                               ', bos=False)

[168]

In [ ]:
t_fast.encode('                               ', add_special_tokens=False)

[168]

In [ ]:
len(encoded)

240

In [ ]:
t_fast.encode("                                               ", add_special_tokens=False)

[153, 168]

In [ ]:
original.encode("                                               ", bos=False)

[168, 153]

In [ ]:
t_slow.encode("                                               ", add_special_tokens=False)

[168, 153]

In [ ]:
len("                                               ")

47

In [ ]:
len('                               '), len('                ')

(31, 16)

In [ ]:
slow_encoded = t_slow.encode(code)
print(slow_encoded)

[2, 809, 11092, 235250, 235265, 8736, 685, 11092, 792, 89846, 108, 809, 11092, 235250, 235265, 3744, 5815, 685, 11092, 16437, 235266, 108, 809, 53649, 109, 1638, 4671, 2720, 7761, 235278, 51568, 235265, 28132, 1245, 108, 141, 1293, 33826, 235278, 1053, 1245, 108, 145, 1053, 235265, 7363, 589, 11092, 16437, 235266, 235265, 3744, 5815, 235278, 108, 149, 235281, 16833, 235283, 2195, 235298, 1259, 235283, 26154, 235265, 3327, 824, 108, 149, 9766, 48020, 235298, 2222, 1192, 664, 16833, 235283, 2195, 235298, 1259, 235283, 5490, 12990, 108, 145, 235275, 108, 145, 1053, 235265, 8578, 235298, 1701, 589, 892, 108, 147, 235281, 14214, 235265, 3028, 824, 108, 147, 235281, 235267, 85897, 477, 235265, 6001, 824, 108, 147, 235281, 8840, 4996, 553, 31524, 574, 4996, 553, 620, 235259, 1014, 235265, 4405, 235262, 235281, 108, 145, 235307, 109, 141, 1293, 2121, 235298, 7591, 480, 183853, 235278, 1053, 1245, 108, 145, 1053, 235265, 20658, 885, 14214, 235265, 3028, 235281, 780, 575, 108, 161, 95062, 8736, 

In [ ]:
fast_encoded == encoded, slow_encoded == encoded

(False, True)

In [ ]:
manually_manipulated_fast = AutoTokenizer.from_pretrained("/data/models/gemma/from_hub/tokenizer_bug/recurrentgemma-2b")

In [ ]:
manually_manipulated_fast.encode("                                               ", add_special_tokens=False)

[153, 168]